In [1]:
from scqbf.scqbf_evaluator import *
from scqbf.scqbf_solution import *
from scqbf.scqbf_ga import *
from scqbf.scqbf_instance import *

import pandas as pd
import numpy as np
import pickle

In [2]:
# instance_paths = [(f"instances/gen{i}/instance{j}.txt", i, j) for i in range(1, 4) for j in range(1, 6)]
instance_paths = [(f"instances/gen1/instance4.txt", 1, 4)]

In [3]:
import random

# Set population size
pop_size_1 = 100 
pop_size_2 = random.randint(100, 1000) # 

# Set mutation rate
mutation_rate_1 = 1/100
mutation_rate_2 = 1/50

In [ ]:
def run_experiment(config: dict, pop_size, mutation_rate) -> pd.DataFrame:
    results = []
    exp_num = 0

    for instance_path, gen, inst in instance_paths:
        # Read Instance
        instance = read_max_sc_qbf_instance(instance_path)
        print(f"{exp_num}: {inst}th instance of generation strategy {gen}. Path: {instance_path}")
        exp_num += 1
        
        # Set stop criteria values
        if config.get('stop_criteria', 'time') == 'time':
            time_limit = 60*30 # 30 minutes
            n_generations = None
        elif config.get('stop_criteria', 'time') == 'generations':
            time_limit = None
            n_generations = 1000

        # Run Genetic Algorithm
        ga = ScQbfGeneticAlgorithm(
            instance, 
            time_limit_secs=time_limit, 
            generations=n_generations, 
            pop_size=pop_size, 
            mutation_rate=mutation_rate, 
            debug=True, 
            config=config
        )
        best_solution = ga.solve()
        
        # Evaluate and save metrics
        evaluator = ScQbfEvaluator(instance)
        results.append({
            'gen': gen,
            'inst': inst,
            'n': instance.n,
            'best_objective': round(evaluator.evaluate_objfun(best_solution), 4),
            'coverage': evaluator.evaluate_coverage(best_solution),
            'time_taken': round(ga.solve_time),
            'n_generations': ga.current_generation,
        })
        
        # Print values
        last_result = results[-1]
        print(f"\tBest objective value: {last_result['best_objective']:.4f}")
        print(f"Selected elements: {best_solution.elements}")
        print(f"\tCoverage: {last_result['coverage']:.2%}")
        print(f"\tGenerations executed: {last_result['n_generations']}")
        print()

    df = pd.DataFrame(results)
    return df

## Configuração 1 - PADRÃO:
GA com população de tamanho 'pop_size_1', taxa de mutação 'mutation_rate_1' e construção aleatória da população

In [5]:
config = {
    'stop_criteria': 'time' # Options: 'time', 'generations'
}
results_df = run_experiment(config, pop_size_1, mutation_rate_1)
results_df.to_csv('results/experiment_results_1.csv', index=False)

display(results_df)

0: 4th instance of generation strategy 1. Path: instances/gen1/instance4.txt
Generation 0: BestSolution = ScQbfSolution(elements=[0, 3, 4, 7, 10, 11, 12, 15, 18, 22, 23, 24], _last_objfun_val=161.04000000000002)
Generation 2: BestSolution = ScQbfSolution(elements=[0, 3, 4, 7, 10, 11, 12, 15, 18, 23, 24], _last_objfun_val=175.25000000000006)
Generation 3: BestSolution = ScQbfSolution(elements=[0, 3, 4, 7, 10, 11, 12, 15, 18, 19, 20, 22, 23, 24], _last_objfun_val=185.72000000000003)
Generation 4: BestSolution = ScQbfSolution(elements=[0, 3, 4, 7, 10, 11, 12, 15, 17, 20, 23, 24], _last_objfun_val=211.0300000000001)
Generation 5: BestSolution = ScQbfSolution(elements=[0, 3, 4, 7, 10, 11, 12, 13, 15, 17, 18, 19, 20, 23, 24], _last_objfun_val=243.18999999999994)
Generation 10: BestSolution = ScQbfSolution(elements=[0, 3, 4, 5, 7, 8, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 23, 24], _last_objfun_val=249.27999999999986)
Generation 15: BestSolution = ScQbfSolution(elements=[0, 3, 4, 5, 6, 7, 8, 

,gen,inst,n,best_objective,coverage,time_taken,n_generations
0,1,4,25,261.06,1.0,120,26986


## Configuração 2 - PADRÃO + UNIFORM CROSSOVER:
GA com população de tamanho 'pop_size_1', taxa de mutação 'mutation_rate_1' e construção aleatória da população, uniform crossover

In [6]:
config = {
    'stop_criteria': 'time', # Options: 'time', 'generations'
    'crossover_type': 'uniform' # Options: 'default', 'uniform' (uses 'default' if omitted)
}
results_df = run_experiment(config, pop_size_1, mutation_rate_1)
results_df.to_csv('results/experiment_results_2.csv', index=False)

display(results_df)

0: 4th instance of generation strategy 1. Path: instances/gen1/instance4.txt
Generation 0: BestSolution = ScQbfSolution(elements=[0, 1, 3, 4, 5, 12, 13, 15, 16, 18, 20, 24], _last_objfun_val=149.64)
Generation 1: BestSolution = ScQbfSolution(elements=[0, 3, 4, 7, 8, 10, 12, 15, 17, 23, 24], _last_objfun_val=183.44000000000005)
Generation 3: BestSolution = ScQbfSolution(elements=[0, 1, 3, 4, 7, 8, 9, 10, 12, 13, 15, 16, 17, 18, 20, 23], _last_objfun_val=193.63999999999996)
Generation 4: BestSolution = ScQbfSolution(elements=[0, 1, 4, 5, 7, 8, 12, 13, 14, 15, 16, 17, 18, 19, 20, 23], _last_objfun_val=196.56000000000006)
Generation 5: BestSolution = ScQbfSolution(elements=[0, 1, 3, 4, 7, 8, 9, 12, 13, 15, 16, 17, 18, 19, 20, 23, 24], _last_objfun_val=210.69)
Generation 6: BestSolution = ScQbfSolution(elements=[0, 1, 3, 4, 5, 7, 8, 12, 13, 14, 15, 16, 18, 19, 20, 23, 24], _last_objfun_val=222.45)
Generation 7: BestSolution = ScQbfSolution(elements=[0, 3, 4, 7, 8, 10, 12, 15, 16, 17, 18, 20

,gen,inst,n,best_objective,coverage,time_taken,n_generations
0,1,4,25,261.06,1.0,120,23080


## Configuração 3 - PADRÃO + Steady-state:
GA com população de tamanho 'pop_size_1', taxa de mutação 'mutation_rate_1' e construção aleatória da população

In [7]:
config = {
    'stop_criteria': 'time', # Options: 'time', 'generations'
    'evolution_mode': 'steady_state' # Options: 'default', 'steady_state'
}
results_df = run_experiment(config, pop_size_1, mutation_rate_1)
results_df.to_csv('results/experiment_results_3.csv', index=False)

display(results_df)

0: 4th instance of generation strategy 1. Path: instances/gen1/instance4.txt
Generation 0: BestSolution = ScQbfSolution(elements=[0, 1, 3, 6, 7, 8, 10, 12, 13, 14, 15, 16, 18, 19, 22, 23], _last_objfun_val=151.98999999999998)
Generation 18: BestSolution = ScQbfSolution(elements=[0, 3, 4, 5, 7, 10, 11, 12, 16, 17, 23, 24], _last_objfun_val=158.97000000000008)
Generation 24: BestSolution = ScQbfSolution(elements=[0, 3, 4, 5, 8, 10, 12, 13, 14, 15, 16, 18, 19, 22, 23, 24], _last_objfun_val=181.07000000000002)
Generation 120: BestSolution = ScQbfSolution(elements=[0, 3, 4, 5, 6, 7, 10, 12, 13, 15, 18, 19, 20, 23, 24], _last_objfun_val=224.51999999999998)
Generation 188: BestSolution = ScQbfSolution(elements=[0, 3, 4, 5, 6, 7, 10, 12, 13, 15, 16, 18, 19, 20, 22, 23, 24], _last_objfun_val=231.79999999999993)
Generation 227: BestSolution = ScQbfSolution(elements=[0, 4, 6, 7, 8, 10, 12, 13, 15, 16, 18, 19, 20, 23, 24], _last_objfun_val=238.82999999999996)
Generation 241: BestSolution = ScQbfSo

,gen,inst,n,best_objective,coverage,time_taken,n_generations
0,1,4,25,261.06,1.0,120,31303
